In [1]:
#Making Necessary imports
#http://ijarcet.org/wp-content/uploads/IJARCET-VOL-1-ISSUE-9-135-139.pdf
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
#STEP1: Convert the face images to face vectors
image_width = 720
image_length = 1280
total_pixels = image_width*image_length

images = 5
variants = 8
total_images = images*variants

face_vector = []

for i in range(1, total_images+1):
    face_image = cv2.cvtColor(cv2.imread("training/" + str(i) + ".jpg"), cv2.COLOR_RGB2GRAY)
#     plt.imshow(face_image, cmap = 'gray', interpolation = 'bicubic')
#     plt.show()
    face_image = face_image.reshape(total_pixels,)
    face_vector.append(face_image)
    
face_vector = np.asarray(face_vector)
face_vector = face_vector.transpose()

print(face_vector.shape)
print(face_vector)

(921600, 40)
[[230 234 224 ... 215 205 200]
 [230 234 224 ... 215 205 200]
 [230 234 224 ... 215 206 200]
 ...
 [154  45  31 ... 131 115 121]
 [155  44  32 ... 131 116 129]
 [155  43  32 ... 131 117 137]]


In [3]:
#STEP2: Normalize the face vectors by calculating the average face vector and subtracting it from each vector
avg_face_vector = face_vector.mean(axis=1)
avg_face_vector = avg_face_vector.reshape(face_vector.shape[0], 1)
normalized_face_vector = face_vector - avg_face_vector
print(normalized_face_vector)

[[ 21.975  25.975  15.975 ...   6.975  -3.025  -8.025]
 [ 22.125  26.125  16.125 ...   7.125  -2.875  -7.875]
 [ 22.05   26.05   16.05  ...   7.05   -1.95   -7.95 ]
 ...
 [ 86.275 -22.725 -36.725 ...  63.275  47.275  53.275]
 [ 85.925 -25.075 -37.075 ...  61.925  46.925  59.925]
 [ 85.75  -26.25  -37.25  ...  61.75   47.75   67.75 ]]


In [4]:
#STEP3: Calculate the Covariance Matrix or the Sigma
covariance_matrix = np.cov(np.transpose(normalized_face_vector))
# covariance_matrix = np.transpose(normalized_face_vector).dot(normalized_face_vector)
print(covariance_matrix)

[[ 1984.7979128    344.90924566  1049.80024931 ...  -972.62674781
   -202.63888962  -725.16929237]
 [  344.90924566  2664.60808719   313.51728208 ...  -212.88955595
   -780.91583095  -572.19202933]
 [ 1049.80024931   313.51728208  2586.49051488 ... -1218.48776572
   -228.08512048  -504.07805965]
 ...
 [ -972.62674781  -212.88955595 -1218.48776572 ...  1887.47367789
    294.54730847   709.0548827 ]
 [ -202.63888962  -780.91583095  -228.08512048 ...   294.54730847
   2368.80268429   237.26077386]
 [ -725.16929237  -572.19202933  -504.07805965 ...   709.0548827
    237.26077386  2272.19503562]]


In [5]:
#STEP4: Calculate Eigen Vectors
eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)

In [6]:
#STEP5: Select the K best Eigen Faces, K < M
print(eigen_vectors.shape)
k = 20
k_eigen_vectors = eigen_vectors[0:k, :]
print(k_eigen_vectors.shape)

(40, 40)
(20, 40)


In [7]:
#STEP6: Convert lower dimensionality K Eigen Vectors to Original Dimensionality
eigen_faces = k_eigen_vectors.dot(np.transpose(normalized_face_vector))
print(eigen_faces.shape)

(20, 921600)


In [8]:
# STEP7: Represent Each eigen face as combination of the K Eigen Vectors
# weights = eigen_faces.dot(normalized_face_vector)
weights = np.transpose(normalized_face_vector).dot(np.transpose(eigen_faces))
print(weights)

[[ 4.32967094e+08 -6.20076519e+08  2.30090689e+08 -3.46128769e+08
   5.64386234e+08  2.54578597e+08 -2.52444132e+08 -6.51178172e+08
  -1.42701122e+08 -1.65555104e+08 -9.05959119e+08 -1.19022261e+09
  -1.10352393e+09 -6.06536595e+08 -3.35422706e+08  3.41754489e+08
  -2.05373217e+08 -6.85550626e+08 -4.31517069e+08  9.85829866e+08]
 [-5.81836777e+08 -1.28609396e+09  3.50461032e+08 -7.02799360e+08
   2.34335850e+08  4.69236536e+07 -1.38755484e+09 -7.53402918e+08
   7.39962948e+08 -4.99722179e+08 -2.16117787e+08 -1.78934454e+08
   6.70159626e+08  6.79056362e+08 -5.41564772e+08  7.52322137e+08
   3.31182727e+08 -6.74918247e+08  1.44026295e+08  8.10493584e+08]
 [-2.90210143e+08 -7.59747343e+08 -5.21646738e+07 -2.07927137e+08
   1.10519288e+08  6.69369945e+07 -2.30612957e+08 -7.39749741e+08
   3.48543171e+07 -2.28994953e+06  1.21506033e+07 -1.43395497e+09
  -1.36668893e+09 -1.97927683e+08 -5.82331677e+08 -4.13924956e+08
  -5.20974874e+08 -1.46957922e+09 -5.11632182e+08  1.28574942e+08]
 [-8.89

In [9]:
#STEP8: Testing Phase
test_add = "testing/" + "8" + ".jpg"
test_img = cv2.imread(test_add)
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2GRAY)


test_img = test_img.reshape(total_pixels, 1)
test_normalized_face_vector = test_img - avg_face_vector
test_weight = np.transpose(test_normalized_face_vector).dot(np.transpose(eigen_faces))


index =  np.argmin(np.linalg.norm(test_weight - weights, axis=1))    
     
if(index>=0 and index <5):
    print("Nandan Raj")
if(index>=5 and index<10):
    print("Saurabh Mishra")
if(index>=10 and index<15):
    print("Gagan Ganpathy")
if(index>=15 and index<20):
    print("Badnena Upendra")
if(index>=20 and index<25):
    print("Sai Charan")
if(index>=25 and index<30):
    print("Luv NA")
if(index>=30 and index<35):
    print("Manavdeep Singh")
if(index>=35 and index<40):
    print("Anagh Rao")

Anagh Rao
